In [1]:
!python3 prepare_data2.py shakespeare_input.txt shake \\n\\n+ -m 400

Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 400 characters...
28645 sequences remaining.
Longest remaining sequence has length 399.
Removing length-0 sequences...
28645 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...

Serialized 26800 sequences...
Serialized 26900 sequences...
Serialized 27000 sequences...
Serialized 27100 sequences...
Serialized 27200 sequences...
Serialized 27300 sequences...
Serialized 27400 sequences...
Serialized 27500 sequences...
Serialized 27600 sequences...
Serialized 27700 sequences...
Serialized 27800 sequences...
Serialized 27900 sequences...
Serialized 28000 sequences...
Serialized 28100 sequences...
Serialized 28200 sequences...
Serialized 28300 sequences...
Serialized 28400 sequences...
Serialized 28500 sequences...
Serialized 28600 sequences...


In [54]:
import pickle

import tensorflow as tf
from prepare_data2 import parse_seq
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import os

# data
bs = 128


data = tf.data.TFRecordDataset("shake.tfrecords")
data = data.map(lambda x: parse_seq(x))

batched_categorical_data = data.padded_batch(batch_size=128, padded_shapes=400,padding_values=0, drop_remainder=True)

vocab = pickle.load(open("shake_vocab", mode="rb"))

vocab_size = len(vocab)
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

ch_to_ind = {ch: ind for (ind, ch) in vocab.items()}

print(vocab_size)
print(vocab)




70
{'v': 3, 'z': 4, 'h': 5, 'o': 6, 'g': 7, 'a': 8, 'N': 9, ' ': 10, 'q': 11, 'W': 12, 'd': 13, 'n': 14, 'b': 15, 'V': 16, 'c': 17, ':': 18, 'F': 19, 'Y': 20, 'G': 21, '$': 22, "'": 23, 'J': 24, 'p': 25, 'l': 26, 'X': 27, 'A': 28, 'e': 29, 'D': 30, 'K': 31, 'P': 32, ';': 33, 'Z': 34, 'R': 35, 'j': 36, '!': 37, 'Q': 38, 't': 39, '.': 40, 'y': 41, 'I': 42, 'u': 43, 'r': 44, 'O': 45, 'T': 46, 'k': 47, 'H': 48, 'B': 49, 'm': 50, '\n': 51, 'x': 52, 'S': 53, 'M': 54, 'f': 55, 'E': 56, 'C': 57, 'L': 58, '?': 59, 'U': 60, '&': 61, '3': 62, 's': 63, '[': 64, ',': 65, '-': 66, 'i': 67, ']': 68, 'w': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}


In [26]:
for num, element in enumerate(data):
    if(num > 1):
        break;
    
    print(element)
    print(" ====== Creating Labels by taking slices ==========")
    print(element[1:])

tf.Tensor(
[ 1 19 67 44 63 39 10 57 67 39 67  4 29 14 18 51 49 29 55  6 44 29 10 69
 29 10 25 44  6 17 29 29 13 10  8 14 41 10 55 43 44 39  5 29 44 65 10  5
 29  8 44 10 50 29 10 63 25 29  8 47 40  2], shape=(62,), dtype=int32)
 ====== Creating Labels by taking slices ==========
tf.Tensor(
[19 67 44 63 39 10 57 67 39 67  4 29 14 18 51 49 29 55  6 44 29 10 69 29
 10 25 44  6 17 29 29 13 10  8 14 41 10 55 43 44 39  5 29 44 65 10  5 29
  8 44 10 50 29 10 63 25 29  8 47 40  2], shape=(61,), dtype=int32)
tf.Tensor([ 1 28 26 26 18 51 53 25 29  8 47 65 10 63 25 29  8 47 40  2], shape=(20,), dtype=int32)
 ====== Creating Labels by taking slices ==========
tf.Tensor([28 26 26 18 51 53 25 29  8 47 65 10 63 25 29  8 47 40  2], shape=(19,), dtype=int32)


In [27]:
def create_label(ds):
    return ds[1:];

all_label_data = data.map(create_label)
batched_label_data = all_label_data.padded_batch(batch_size=128, padded_shapes=400, padding_values=0, drop_remainder=True)

In [29]:
def onehotencode(ds):
    new_data = tf.one_hot(indices = ds, depth = vocab_size)
    return new_data;

onehot_encoded_batch_data = batched_categorical_data.map(onehotencode)
onehot_encoded_label_data = batched_label_data.map(onehotencode)

In [69]:
units = 512
opt = tf.optimizers.Adam()
n_time_steps = 20

In [35]:
def build_model(vocab_size, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [36]:
model = build_model(
  vocab_size = vocab_size,
  rnn_units=400,
  batch_size=128)

In [57]:
# Directory where the checkpoints will be saved
checkpoint_dir = './train_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [70]:
def custom_masked_loss1(one_batch_data, max_len, logits, labels):

    non_zero_counts = tf.math.count_nonzero(input=one_batch_data, axis = 1, dtype=tf.dtypes.float32)

    #Do -1 as the last element of each sequence isn’t used as input.
    non_zero_counts = non_zero_counts - 1

    #Creating the mask
    mask = tf.sequence_mask(non_zero_counts, max_len, dtype=tf.dtypes.float32 )

    #Calculate loss for each element of the batch and timestep
    loss_tnsr = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

    #Create a mask based on non-zero sequence
    masked_loss = tf.multiply(loss_tnsr, mask)

    summed_loss_per_sequence_of_batch = tf.reduce_sum(masked_loss, axis = 1)

    average_loss_per_sequence_of_batch = tf.divide(summed_loss_per_sequence_of_batch, non_zero_counts)

    #Since the loss is already masked and averaged per sequence
    average_masked_loss_per_batch = tf.reduce_mean(average_loss_per_sequence_of_batch, axis = None)

    return average_masked_loss_per_batch;

In [71]:
@tf.function
def train_step(inp, target, one_batch_data, max_len):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = custom_masked_loss1(one_batch_data, max_len, logits = predictions, labels=target)
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

    return loss;

In [ ]:
# Training step
EPOCHS = 20
max_len = 400
for epoch in range(EPOCHS):

    hidden = model.reset_states()

    for (batch_n, (onehot_inp, categ_inp, categ_target)) in enumerate(zip(onehot_encoded_batch_data, batched_categorical_data, batched_label_data)):
        loss = train_step(inp = onehot_inp, target = categ_target, one_batch_data = categ_inp, max_len = max_len)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.253420352935791
Epoch 1 Batch 100 Loss 2.685504198074341
Epoch 1 Batch 200 Loss 2.3132948875427246
Epoch 1 Loss 2.3348
Epoch 2 Batch 0 Loss 2.421513557434082
Epoch 2 Batch 100 Loss 2.2595455646514893
Epoch 2 Batch 200 Loss 2.1168062686920166
Epoch 2 Loss 2.1558
Epoch 3 Batch 0 Loss 2.1728715896606445
Epoch 3 Batch 100 Loss 2.1126186847686768
Epoch 3 Batch 200 Loss 1.9706356525421143
Epoch 3 Loss 2.0233
Epoch 4 Batch 0 Loss 1.9730671644210815
Epoch 4 Batch 100 Loss 1.9916248321533203
Epoch 4 Batch 200 Loss 1.8525691032409668
Epoch 4 Loss 1.9070
Epoch 5 Batch 0 Loss 1.8313437700271606
Epoch 5 Batch 100 Loss 1.9065754413604736
Epoch 5 Batch 200 Loss 1.7454496622085571
Epoch 5 Loss 1.8120
Epoch 6 Batch 0 Loss 1.7223670482635498
Epoch 6 Batch 100 Loss 1.8341710567474365
Epoch 6 Batch 200 Loss 1.6543173789978027
Epoch 6 Loss 1.7369
Epoch 7 Batch 0 Loss 1.6398365497589111
